In [ ]:
"""
Team Name: Creatix
Team Members: Siddharth Malkania, Krishan Verma , Rishi Mehrotra
Leaderboard Rank: 117

"""

# This is the notebook used for making the inferences using the model trained.
# Inference Notebook for Soil Classification

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# GPU optimization (optional)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print(f"GPU acceleration enabled: {len(physical_devices)} GPU(s) found")
else:
    print("No GPU found, using CPU")

# Configuration
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = 4

# Paths (update as needed)
MODEL_PATH = '/kaggle/input/your-model-directory/model.h5'
TEST_DIR = '/kaggle/input/soilcl/soil_classification-2025/test'
TEST_CSV = '/kaggle/input/soilcl/soil_classification-2025/test_ids.csv'

# Prepare test data
test_df = pd.read_csv(TEST_CSV)

# If images need to be converted to JPG, use the convert_to_jpg function from your training notebook

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=TEST_DIR,
    x_col='image_id',
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

# Rebuild model architecture (must match training)
def build_model():
    base_model = DenseNet121(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Load trained weights
model = build_model()
model.load_weights(MODEL_PATH)
print("Loaded trained model weights.")

# Predict on test data
test_generator.reset()
preds = model.predict(test_generator, steps=int(np.ceil(test_generator.samples/BATCH_SIZE)))
predicted_classes = np.argmax(preds, axis=1)

# Map integer labels to class names (update mapping as per your training)
class_indices = {'Clay soil': 0, 'Red Soil': 1, 'Sandy Soil': 2, 'Black Soil': 3}
idx_to_class = {v: k for k, v in class_indices.items()}
predicted_labels = [idx_to_class[idx] for idx in predicted_classes]

# Prepare submission
submission = pd.DataFrame({
    'image_id': test_df['image_id'],
    'soil_type': predicted_labels
})
submission.to_csv('submission.csv', index=False)
print("Inference complete. Submission file saved as submission.csv")
